In [1]:
!pip install PyYAML


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

In [3]:
# Load YAML file
with open('data.yaml', mode='r') as f:
    data_yaml = yaml.load(f, Loader=SafeLoader)

labels = data_yaml['names']
print(labels)

['person', 'car', 'chair', 'bottle', 'pottedplant', 'bird', 'dog', 'sofa', 'bicycle', 'horse', 'boat', 'motorbike', 'cat', 'tvmonitor', 'cow', 'sheep', 'aeroplane', 'train', 'diningtable', 'bus']


In [4]:
# Load YOLO Model

yolo = cv2.dnn.readNetFromONNX('./YOLOV5_Model/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

print(yolo)

< cv2.dnn.Net 0000020072B25210>


In [5]:
# Load the Image
img = cv2.imread('./street.jpg')
image = img.copy()

#cv2.imshow('image',image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

row, col, d = image.shape

# Get the Yolo Prediction from that image

#convert image into square image
max_rc = max(row, col)
input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8)
input_image[0:row, 0:col] = image

# get prediction from sqaure
INPUT_WH_YOLO = 640
blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO, INPUT_WH_YOLO),swapRB=True,crop=False)
yolo.setInput(blob)
preds = yolo.forward() # prediction from yolo model

#cv2.imshow('input_image',input_image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [6]:
print(preds.shape)

(1, 25200, 25)


In [7]:
# Non Maximum Supression
# filter detection based on confidence score (0.4) and probability score (0.25)
detections = preds[0]
boxes = []
confidences = []
classes = []

# width and height of the input_image
image_w, image_h = input_image.shape[:2]
x_factor = image_w/INPUT_WH_YOLO
y_factor = image_h/INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4] #confidence of detection
    if confidence > 0.4:
        class_score = row[5:].max() # max prob of 20 object
        class_id = row[5:].argmax() # get index position of whch maximum prob occur

        if class_score > 0.25:
            cx, cy, w, h = row[0:4]

            # constract bndbox from 4 val left, top, width, heigth
            left = int((cx - 0.5*w)*x_factor)
            top = int((cy - 0.5*h)*y_factor)
            width = int(w*x_factor)
            height = int(h*y_factor)

            box = np.array([left, top, width, height])

            # append values
            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)

#clean
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

#NMS
index = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.25, 0.15).flatten()

In [8]:
len(index)

9

In [9]:
# Draw the bounding box

for ind in index:
    x,y,w,h = boxes_np[ind] # extract bnd box
    bb_conf = int(confidences_np[ind]*100)
    classes_id = classes[ind]
    class_name = labels[classes_id]

    text = f'{class_name}: {bb_conf}%'

    cv2.rectangle(image,(x,y),(x+w, y+h),(0,255,0),2)
    cv2.rectangle(image,(x,y-30),(x+w,y),(255,255,255),-1)
    
    cv2.putText(image,text,(x,y-10),cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)

    print(text)
    

car: 83%
bus: 82%
bus: 74%
car: 64%
car: 62%
person: 58%
person: 46%
person: 45%
person: 43%


In [ ]:
cv2.imshow('original',img)
cv2.imshow('yolo_prediction',image)
cv2.waitKey(0)
cv2.destroyAllWindows()